In [17]:
from sklearn.metrics import roc_auc_score
import numpy as np 
import pandas as pd 
import statsmodels.api as sm

df = pd.read_csv('gen_data.csv')
import pickle

ALL_WOE_MODELS = pickle.load(open("ALL_WOE_MODELS.pkl", "rb"))
ALL_FINAL_VARS = pickle.load(open("ALL_FINAL_VARS.pkl", "rb"))
ALL_LR_MODELS   = pickle.load(open("ALL_LR_MODELS.pkl", "rb"))


df['LR_PROB'] = np.nan

segments = ["1. Rich/Secured", "2. Prime Unsecured", "3. Mass Unsecured"]


for seg in segments:
    print(f"Scoring segment: {seg}")
    # Lấy mask để đảm bảo dùng chung vị trí
    mask = (df['SEGMENT'] == seg)
    df_seg = df[mask].copy()

    woe_dict = ALL_WOE_MODELS[seg]
    var_list = ALL_FINAL_VARS[seg]
    lr_model = ALL_LR_MODELS[seg]

    X_woe = pd.DataFrame(index=df_seg.index) # Khởi tạo kèm index gốc
    for feature in var_list:
        X_woe[feature] = woe_dict[feature].transform(df_seg[feature], metric="woe")

    X_const = sm.add_constant(X_woe)
    
    # Dùng .values ở đây để tránh lệch index
    df.loc[mask, 'LR_PROB'] = lr_model.predict(X_const).values

# Calculate overall Gini
y_true = df['BAD_NEXT_12M']
y_prob = df['LR_PROB']

auc = roc_auc_score(y_true, y_prob)
gini = 2*auc - 1

print(f"\n>>> GINI TOÀN BỘ (3 segment LR gộp lại): {gini:.4f}")

Scoring segment: 1. Rich/Secured
Scoring segment: 2. Prime Unsecured
Scoring segment: 3. Mass Unsecured

>>> GINI TOÀN BỘ (3 segment LR gộp lại): 0.8735


In [18]:
df['SEGMENT'].unique()

array(['3. Mass Unsecured', '2. Prime Unsecured', '1. Rich/Secured'],
      dtype=object)

In [19]:
print("WOE keys:", ALL_WOE_MODELS.keys())

WOE keys: dict_keys(['2. Prime Unsecured', '1. Rich/Secured', '3. Mass Unsecured'])


In [20]:
df.groupby("SEGMENT")['LR_PROB'].apply(lambda x: x.isna().mean())


SEGMENT
1. Rich/Secured       0.0
2. Prime Unsecured    0.0
3. Mass Unsecured     0.0
Name: LR_PROB, dtype: float64

In [21]:
X_woe.isna().sum()

MAX_DPD_12M_OBS      0
N_AVG_DEPOSIT_12M    0
CBAL_TO_INC_12MON    0
dtype: int64

In [22]:
print("SEGMENT counts:\n", df['SEGMENT'].value_counts())
print("\nNaN ratio by segment:\n", df.groupby("SEGMENT")['LR_PROB'].apply(lambda x: x.isna().mean()))
print("\nExample NaN rows:\n", df[df['LR_PROB'].isna()].head())


SEGMENT counts:
 SEGMENT
3. Mass Unsecured     437372
1. Rich/Secured       421292
2. Prime Unsecured    279495
Name: count, dtype: int64



NaN ratio by segment:
 SEGMENT
1. Rich/Secured       0.0
2. Prime Unsecured    0.0
3. Mass Unsecured     0.0
Name: LR_PROB, dtype: float64

Example NaN rows:
 Empty DataFrame
Columns: [SOCIF, C_GIOITINH, TRINHDO, TTHONNHAN, SOHUUNHA, NHANVIENBIDV, INHERENT_RISK, REF_MONTH, REF_DAY, year, BASE_AUM, CURRENT_RISK, TUOI, SNAPSHOT_DATE, INCOME, CBAL, CBALORG, AFLIMT_MAX, AFLIMT_MIN, AFLIMT_AVG, CBAL_AVG, CBAL_MAX, CBAL_MIN, COLLATERAL_VALUE, LTV, N_AVG_DEPOSIT_12M, N_AVG_DEPOSIT_6M, N_AVG_DD_12M, N_AVG_CD_12M, FLAG_SALARY_ACC, FLAG_DEPOSIT, CBAL_SHORTTERM_LOAN, CBAL_LONGTERM_LOAN, HAS_SHORTTERM_LOAN, HAS_LONGTERM_LOAN, DURATION_MAX, REMAINING_DURATION_MAX, TIME_TO_OP_MAX, RATE_AVG, PURCOD_MAX, PURCOD_MIN, MAX_DPD_12M, MAX_DPD_12M_OBS, AVG_OD_DPD_12M, SUM_ALL_OD_12M, BAD_CURRENT, XULYNO, MAX_NHOMNOCIC, N_AVG_OVERDUE_CBAL_12M, CBAL_TO_INC_12MON, REAL_GDP_GROWTH_12M, BAD_NEXT_12M, SAMPLE_TYPE, SEGMENT, LR_PROB]
Index: []

[0 rows x 55 columns]


In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

# ============================================================
# 1. DANH SÁCH FILE KẾT QUẢ TỪ CÁC SEGMENT
# ============================================================
# Giả sử bạn có 2 segments, thêm tên file vào list này
segment_files = [
    r'C:\Users\PC\Documents\GitHub\Khoa-luan\Log reg\seg1_result.parquet',
    r"C:\Users\PC\Documents\GitHub\Khoa-luan\Log reg\seg2_result.parquet",
    r"C:\Users\PC\Documents\GitHub\Khoa-luan\Log reg\seg3_result.parquet" 
]

# ============================================================
# 2. ĐỌC VÀ GỘP DỮ LIỆU (STACKING)
# ============================================================
df_list = []
for f in segment_files:
    try:
        temp_df = pd.read_parquet(f)
        df_list.append(temp_df)
        print(f"-> Đã đọc: {f} | Shape: {temp_df.shape}")
    except Exception as e:
        print(f"Lỗi không đọc được file {f}: {e}")

# Gộp lại thành 1 DataFrame duy nhất
df_total = pd.concat(df_list, axis=0, ignore_index=True)
print(f"\n=> TỔNG SỐ LƯỢNG MẪU (ALL SEGMENTS): {df_total.shape}")

# ============================================================
# 3. HÀM TÍNH GINI
# ============================================================
def calculate_gini(y_true, y_prob):
    try:
        # Gini = 2 * AUC - 1
        auc = roc_auc_score(y_true, y_prob)
        gini = 2 * auc - 1
        return gini
    except:
        return np.nan

# ============================================================
# 4. TÍNH GINI CHUNG (OVERALL) THEO TỪNG TẬP DỮ LIỆU
# ============================================================
print("\n" + "="*40)
print("   KẾT QUẢ GINI CHUNG (ALL SEGMENTS)")
print("="*40)

# Duyệt qua từng loại tập dữ liệu
summary_gini = []

for dtype in ['TRAIN', 'OOS', 'OOT']:
    # Lọc dữ liệu theo loại (Train/OOS/OOT) của TOÀN BỘ các segment gộp lại
    subset = df_total[df_total['DATA_TYPE'] == dtype]
    
    if len(subset) > 0:
        # Tính Gini
        gini_score = calculate_gini(subset['BAD_NEXT_12M'], subset['PREDICTED_PROB'])
        
        print(f"Dataset: {dtype:<10} | Records: {len(subset):<7} | Gini: {gini_score:.4f}")
        
        summary_gini.append({
            'DATA_TYPE': dtype,
            'Records': len(subset),
            'Overall_Gini': gini_score
        })
    else:
        print(f"Dataset: {dtype:<10} | Không có dữ liệu")

# ============================================================
# 5. (TÙY CHỌN) KIỂM TRA GINI CHI TIẾT LẠI TỪNG SEGMENT
# ============================================================
# Bước này để đảm bảo khi gộp vào không bị sai sót so với lúc chạy lẻ
print("\n" + "="*40)
print("   BREAKDOWN THEO TỪNG SEGMENT")
print("="*40)

pivot_gini = df_total.groupby(['SEGMENT', 'DATA_TYPE']).apply(
    lambda x: calculate_gini(x['BAD_NEXT_12M'], x['PREDICTED_PROB'])
).reset_index(name='GINI')

# Pivot table cho dễ nhìn
report_table = pivot_gini.pivot(index='SEGMENT', columns='DATA_TYPE', values='GINI')
print(report_table)

# ============================================================
# 6. XUẤT KẾT QUẢ CUỐI CÙNG (MASTER FILE)
# ============================================================
# File này dùng để đẩy sang hệ thống chấm điểm hoặc làm báo cáo tổng
df_total.to_parquet("FINAL_MODEL_SCORED_ALL.parquet", index=False)

-> Đã đọc: C:\Users\PC\Documents\GitHub\Khoa-luan\Log reg\seg1_result.parquet | Shape: (787973, 5)
-> Đã đọc: C:\Users\PC\Documents\GitHub\Khoa-luan\Log reg\seg2_result.parquet | Shape: (296611, 5)
-> Đã đọc: C:\Users\PC\Documents\GitHub\Khoa-luan\Log reg\seg3_result.parquet | Shape: (655653, 5)

=> TỔNG SỐ LƯỢNG MẪU (ALL SEGMENTS): (1740237, 5)

   KẾT QUẢ GINI CHUNG (ALL SEGMENTS)
Dataset: TRAIN      | Records: 1137807 | Gini: 0.6702
Dataset: OOS        | Records: 300317  | Gini: 0.6875
Dataset: OOT        | Records: 302113  | Gini: 0.6863

   BREAKDOWN THEO TỪNG SEGMENT


C:\Users\PC\AppData\Local\Temp\ipykernel_15376\1668024396.py:79: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pivot_gini = df_total.groupby(['SEGMENT', 'DATA_TYPE']).apply(


DATA_TYPE                       OOS       OOT     TRAIN
SEGMENT                                                
1. Prime (High Liquidity)  0.692659  0.692551  0.690820
2. Standard (Low DTI)      0.483297  0.469779  0.468570
3. Subprime (High Risk)    0.456057  0.446782  0.451203
